## Ferramentas de análise das propriedades da proteína

1. [main()](#main)
2. [tag_as_key(dictionary, uniprots)](#tag_as_key)
3. [add_info_to_dictionary(dictionary, uniprots)](#add_info_to_dictionary)

Os resultados desta fase podem ser encontrados [aqui](uniprot_results.html).

No final de extrair esta informação, atualizámos a informação retirada do NCBI. A tabela que esta informação conjunta está [aqui](ncbi_uniprot_results.html).


<a name="main"></a>
A resolução desta fase encontra-se no ficheiro [third.py](https://github.com/vitorenesduarte/bionella/tree/master/third.py).
```python
import util.www as www
import util.rw as rw

ncbi_json_path = ".ncbi.json"
ncbi_uniprot_json_path = ".ncbi_uniprot.json"
ncbi_uniprot_diff_json_path = ".ncbi_uniprot_diff.json"

dictionary = rw.read_json(ncbi_json_path)

uniprot_ids = set()
for tag in dictionary:
    if "uniprot_id" in dictionary[tag]:
        uniprot_id = dictionary[tag]["uniprot_id"]
        uniprot_ids.add(uniprot_id)

uniprot_ids = list(uniprot_ids)

uniprots = www.fetch_uniprots(uniprot_ids)
uniprots = tag_as_key(dictionary, uniprots)

(diff, dictionary) = add_info_to_dictionary(
    dictionary,
    uniprots
)
rw.write_json(diff, ncbi_uniprot_diff_json_path)
rw.write_json(dictionary, ncbi_uniprot_json_path)
```

Todas as funções desta fase já foram utilizadas nas fases anteriores com excepção de:
- `tag_as_key`: altera as chaves do dicionário `uniprots` de __uniprot_id__ para __locus_tag__
- `add_info_to_dictionary`: adiciona ao dicionário com informação do NCBI, a informação que retirámos da UniProt:
    - __EC_number__
    - __translation__
    - __length__
    - __accessions__
    - __status__
    - __organism__
    - __short_name__
    - __product__
    - __comment_functions__
    - __cofactors__
    - __pdbs__
    - __pathologies__
    - __molecular_functions__
    - __biological_processes__
    - __locations__
    - __mass__
    - __modified_residues__
    
Como as três primeiras propriedades (__EC_number__, __translation__ e __length__) já se encontravam nesse dicionário, verificamos se os seus valores eram o mesmo: se encontrássemos um valor diferente, atualizávamos o dicionário, mas guardávamos essa informação em `diff` para uma posterior visualização na [tabela](ncbi_uniprot_results.html).

<a name="tag_as_key"></a>
```python
def tag_as_key(dictionary, uniprots):
    """
    Retorna um dicionário com os resultados da uniprot,
    em que as chaves são locus tag em vez de uniprot_id.
    """
    d = {}

    for tag in dictionary:
        if "uniprot_id" in dictionary[tag]:
            uniprot_id = dictionary[tag]["uniprot_id"]

            info = uniprots[uniprot_id]
            d[tag] = info

    return d
```

<a name="add_info_to_dictionary"></a>
```python
def add_info_to_dictionary(dictionary, uniprots):
    """
    Esta função adiciona ao dicionário com informação do NCBI
    a informação retirada do site da uniprot.
        - EC_number
        - translation
        - length
        - accessions
        - status
        - organism
        - short_name
        - product
        - comment_functions
        - cofactors
        - pdbs
        - pathologies
        - molecular_functions
        - biological_processes
        - locations
        - mass
        - modified_residues
    """

    # propriedades que já tinhamos:
    old_properties = [
        "EC_number",
        "translation",
        "length"
    ]

    new_properties = [
        "accessions",
        "status",
        "organism",
        "short_name",
        "product",
        "comment_functions",
        "cofactors",
        "pdbs",
        "pathologies",
        "molecular_functions",
        "biological_processes",
        "locations",
        "mass",
        "modified_residues"
    ]

    diff = {}

    for tag in sorted(dictionary.keys()):
        if tag in uniprots:
            for p in old_properties:
                new = uniprots[tag][p]
                if p in dictionary[tag]:
                    # se já existe, verifica que é o mesmo
                    current = dictionary[tag][p]
                    if not new == current:
                        # guarda informação sobre o que diferiu

                        if tag in diff:
                            diff[tag].append(p)
                        else:
                            diff[tag] = [p]
                
                # substitui sempre pelo valor da uniprot
                dictionary[tag][p] = new

            for p in new_properties:
                dictionary[tag][p] = uniprots[tag][p]

    return (diff, dictionary)
```